In [1]:
import requests
import time

In [3]:
import sqlite3
 
conn = sqlite3.connect("logs.db")
cursor = conn.cursor()

In [74]:
# # Создание таблицы категорий продуктов 
# cursor.execute("""CREATE TABLE category_goods
#                   (id integer primary key, name text)
#                """)
# # Создание таблицы товаров
# cursor.execute("""CREATE TABLE goods
#                   (id integer primary key autoincrement, name text, category_id integer,
#                    FOREIGN KEY (category_id) REFERENCES category_goods (id))
#                """)
# Создание таблицы пользователей
# cursor.execute("""CREATE TABLE users
#                   (id integer primary key autoincrement, ip text unique, country text)
#                """)
# Создание таблицы посещений
# cursor.execute("""CREATE TABLE visits
#                   (id integer primary key autoincrement, datetime datetime, category_id integer,
#                    good_id integer, user_id integer,
#                    FOREIGN KEY (category_id) REFERENCES category_goods (id),
#                    FOREIGN KEY (good_id) REFERENCES goods (id),
#                    FOREIGN KEY (user_id) REFERENCES users (id))
#                """)
# Создание таблицы истории помещений товаров в корзину
# cursor.execute("""CREATE TABLE cart_history
#                   (id integer primary key, datetime datetime,
#                    good_id integer, user_id integer, amount integer,
#                    FOREIGN KEY (good_id) REFERENCES goods (id),
#                    FOREIGN KEY (user_id) REFERENCES users (id))
#                """)
# # Создание таблицы покупок
# cursor.execute("""CREATE TABLE purchases
#                   (id integer primary key autoincrement, datetime datetime, status BOOLEAN default false, cart_id integer,
#                   FOREIGN KEY (cart_id) REFERENCES cart_history (id))
#                """)


In [5]:
def GetCountryFromIP(ip):
    result = requests.get("http://ip-api.com/json/"+ip+"?fields=country")
    return result.text.split(':')[1].replace('"','').replace("}",'')

In [7]:
f = open('logs.txt')
list_visits_site = []
list_visits_category = []
list_visits_good= []
list_carts = []
list_pay = []
list_success_pay = []
for line in f:
    log = {}
    elems = line.split()
    sections = elems[7].split('/')
    sections.remove('https:')
    while '' in sections:
        sections.remove('')
    log['datetime'] = elems[2] + ' ' + elems[3]
    log['ip'] = elems[6]
    if len(sections) == 3:
        log['category'] = sections[1]
        log['good'] = sections[2]
        list_visits_good.append(log)
    elif len(sections) == 2:
        if 'cart?' in sections[1]:
            params = sections[1].split('&')
            log['amount'] = params[1].split('=')[1]
            log['cart_id'] = params[2].split('=')[1]
            list_carts.append(log)
        elif 'pay?' in sections[1]:
            params = sections[1].split('&')
            log['cart_id'] = params[1].split('=')[1]
            list_pay.append(log)
        elif 'success_pay_' in sections[1]:
            log['cart_id'] = sections[1].split('success_pay_')[1]
            log['status'] = True
            list_success_pay.append(log)
        else:
            log['category'] = sections[1]
            list_visits_category.append(log)
    else:
        list_visits_site.append(log)
print(list_visits_site)
print(list_visits_category)
print(list_visits_good)
print(list_carts)
print(list_pay)
print(list_success_pay)
    

[{'datetime': '2018-08-01 00:01:35', 'ip': '121.165.118.201'}, {'datetime': '2018-08-01 00:03:02', 'ip': '121.165.118.201'}, {'datetime': '2018-08-01 00:05:55', 'ip': '121.165.118.201'}, {'datetime': '2018-08-01 00:12:35', 'ip': '121.165.118.201'}, {'datetime': '2018-08-01 00:30:34', 'ip': '152.107.98.15'}, {'datetime': '2018-08-01 00:30:39', 'ip': '217.89.121.82'}, {'datetime': '2018-08-01 00:30:59', 'ip': '237.206.96.214'}, {'datetime': '2018-08-01 00:37:13', 'ip': '15.227.66.159'}, {'datetime': '2018-08-01 00:37:45', 'ip': '114.0.179.225'}, {'datetime': '2018-08-01 00:38:50', 'ip': '152.107.98.15'}, {'datetime': '2018-08-01 00:41:50', 'ip': '164.30.8.42'}, {'datetime': '2018-08-01 00:55:36', 'ip': '132.241.17.218'}, {'datetime': '2018-08-01 01:03:19', 'ip': '132.241.17.218'}, {'datetime': '2018-08-01 01:06:33', 'ip': '217.89.121.82'}, {'datetime': '2018-08-01 01:18:34', 'ip': '237.206.96.214'}, {'datetime': '2018-08-01 01:19:03', 'ip': '164.30.8.42'}, {'datetime': '2018-08-01 01:20:

In [9]:
print(len(list_success_pay))

737


In [7]:
def GetUser(ip):
    cursor.execute("""select id from users where ip =?""",[(ip)])
    result = cursor.fetchall()
    if len(result) == 0:
            return None
    else:
        return result[0][0]

In [75]:
def GetGood(name,category_id):
    cursor.execute("""select id from goods where name =? and category_id = ?""",[(name),(category_id)])
    result = cursor.fetchall()
    if len(result) == 0:
            return None
    else:
        return result[0][0]

In [17]:
def GetCategoryGood(name):
    cursor.execute("""select id from category_goods where name =?""",[(name)])
    result = cursor.fetchall()
    if len(result) == 0:
            return None
    else:
        return result[0][0]

In [114]:
def AddUser(ip):
    user_id = GetUser(ip)
    if user_id == None:
        try:
            country = GetCountryFromIP(ip)
        except Exception:
            country = None
        cursor.execute("""insert into users (ip,country) values(?,?)""",[(ip),(country)])
    return  GetUser(ip)

In [76]:
for d in list_visits_site:
        user_id = AddUser(d['ip'])
        cursor.execute("""insert into visits (datetime,category_id,good_id,user_id) values(?,?,?,?)""",[(d['datetime']),(None),(None),(user_id)])
        time.sleep(0.5)

1
1
1
1
2
3
4
5
6
2
7
8
8
3
4
7
6
4
7
8
6
2
8
5
3
8
5
4
4
6
9
9
9
9
10
10
10
11
12
13
11
13
13
12
12
12
12
14
14
15
15
15
16
17
18
18
18
18
18
19
20
21
19
19
22
23
24
25
25
26
25
25
26
25
27
27
27
27
28
29
28
28
28
29
29
30
31
32
31
32
32
30
31
30
30
33
33
33
33
33
33
33
34
34
35
35
35
35
36
37
37
37
37
38
39
38
39
38
38
38
38
39
40
40
40
41
40
40
40
41
42
42
42
42
42
43
44
45
44
44
44
45
43
45
43
46
47
46
15
48
47
46
47
48
48
48
49
49
15
50
50
15
15
15
51
52
53
54
55
56
57
58
52
53
59
51
55
51
55
53
53
57
52
55
54
56
57
58
53
58
54
58
59
60
61
60
62
63
61
60
60
61
61
61
64
65
66
65
67
64
64
67
68
67
65
65
66
68
67
68
68
66
66
67
69
70
71
72
73
74
75
76
70
74
77
71
72
75
69
71
73
76
77
69
70
76
77
76
76
77
77
69
72
77
76
69
76
76
69
11
11
78
79
11
11
79
79
78
79
78
80
81
82
83
83
80
81
80
80
82
81
81
81
84
85
84
38
86
86
86
86
86
87
88
89
90
87
88
90
89
87
87
90
89
89
87
89
90
91
92
93
92
93
93
92
92
91
91
94
95
96
94
95
96
96
94
96
95
97
98
99
98
97
100
100
100
101
102
102
102
103
103

655
655
655
655
655
655
656
656
656
656
656
656
656
657
658
659
658
660
660
658
659
659
660
659
660
660
661
662
663
661
663
662
663
663
664
664
665
666
667
598
668
669
670
671
669
671
666
669
672
669
671
667
668
598
671
665
670
671
666
666
671
598
668
666
665
598
598
670
665
598
673
674
675
346
676
673
674
677
673
677
676
346
676
346
678
675
678
678
676
674
679
679
679
680
679
528
681
681
528
528
682
683
682
684
682
685
683
685
682
686
687
688
687
686
687
687
687
689
690
690
689
297
691
691
691
691
691
297
297
692
693
693
692
117
693
693
694
692
695
696
697
696
698
695
698
696
698
699
700
701
700
701
644
699
700
699
702
702
700
644
700
644
644
703
704
268
704
704
705
703
705
268
268
703
703
704
703
268
705
705
706
707
706
707
706
706
706
706
706
706
708
709
16
709
16
708
16
709
16
708
16
710
711
10
711
710
10
710
10
711
710
711
711
710
712
713
714
713
712
714
713
712
712
713
713
714
714
713
712
715
716
475
716
715
717
718
717
718
717
718
717
4
8
4
8
4
4
4
719
719
719
720
721
721
721
72

In [77]:
for d in list_visits_category:
    user_id = AddUser(d['ip'])
    category_id = GetCategoryGood(d['category'])
    if category_id == None:
        cursor.execute("""insert into category_goods (name) values(?)""",[(d['category'])])
        category_id = GetCategoryGood(d['category'])
    cursor.execute("""insert into visits (datetime,category_id,good_id,user_id) values(?,?,?,?)""",[(d['datetime']),(category_id),(None),(user_id)])
    

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
6
4
8
3
5
2
3
6
5
4
8
6
7
7
3
5
4
4
7
3
6
3
5
2
3
3
8
2
5
7
2
6
5
4
2
8
4
4
5
7
7
2
8
6
3
2
2
7
5
5
8
3
5
6
6
6
2
2
4
5
5
8
5
3
3
2
6
5
3
3
4
5
6
6
4
6
8
6
8
4
4
6
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
11
12
13
11
12
12
13
11
11
13
12
13
13
12
11
11
13
11
13
13
13
12
12
12
12
12
12
12
15
15
15
15
15
14
15
15
14
14
14
14
15
14
15
14
15
14
15
14
14
14
14
16
16
16
16
16
17
17
17
17
17
18
18
18
18
18
18
19
21
20
21
20
21
19
21
20
19
21
19
19
19
21
19
20
21
21
19
20
20
19
20
19
19
22
23
24
23
24
22
24
23
24
24
22
24
22
23
22
24
22
22
23
23
26
25
26
25
25
25
26
25
26
25
26
26
25
26
25
25
25
26
26
25
25
25
27
27
27
27
27
27
27
27
27
27
27
27
27
28
28
28
29
28
29
28
28
28
29
29
28
29
29
29
28
29
29
30
32
31
32
31
31
30
31
32
30
32
31
32
30
31
30
31
31
30
30
30
30
30
30
30
30
33
33
33
33
33
33
33
33
33
34
34
34
34
34
34
34
34
34
34
35
35
35
35
35
35
35
35
35
36
36
36
36
36
36
36
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
38
38
39
38
38

231
229
232
228
234
231
236
234
236
235
231
235
232
236
228
228
229
234
228
229
228
228
237
237
237
238
238
237
238
238
237
238
237
238
238
237
237
237
237
237
237
237
237
237
237
242
239
240
240
241
239
242
240
241
242
242
241
242
241
240
241
239
241
240
242
240
242
239
241
239
242
240
241
239
240
240
242
239
241
239
240
240
239
241
241
241
239
241
240
240
240
239
241
240
239
245
142
243
244
142
244
245
243
142
142
244
142
245
244
243
244
142
243
142
142
142
243
245
243
245
245
243
142
243
142
243
142
243
243
246
246
247
178
247
247
178
246
246
247
246
247
246
247
247
178
246
246
178
178
178
178
178
178
178
178
246
178
178
178
178
178
178
248
68
255
250
68
252
249
68
253
252
251
255
248
250
254
254
248
250
253
251
255
249
250
250
251
254
255
253
250
255
68
251
251
68
249
252
255
252
248
251
249
252
253
255
253
254
255
250
255
252
250
249
254
253
68
249
253
68
254
68
254
253
254
68
254
249
68
254
68
68
68
253
253
254
249
249
253
254
254
256
256
256
256
256
256
256
256
256
256
256
256
2

431
431
431
431
431
431
431
431
431
431
431
431
433
432
433
433
432
433
432
432
433
433
432
432
432
432
432
432
433
432
432
434
435
434
435
352
434
435
434
352
435
434
352
434
435
434
352
435
435
435
352
434
434
352
352
352
352
352
352
434
434
352
352
434
434
434
436
437
437
436
437
436
437
437
436
436
437
437
436
436
437
437
436
437
437
437
436
436
437
436
437
437
436
436
437
403
403
403
438
439
438
403
439
438
403
403
439
438
403
438
439
403
438
439
438
438
438
438
403
403
439
438
438
440
440
440
440
440
440
440
440
440
440
445
442
444
445
445
442
441
444
443
441
442
441
445
442
441
445
442
444
443
443
442
443
444
445
444
445
442
443
444
443
443
444
443
443
444
443
444
444
443
444
443
444
444
444
451
450
446
447
451
449
451
450
451
451
451
450
448
447
449
451
448
447
446
450
451
448
450
447
449
450
446
448
450
449
451
451
448
449
449
448
446
450
449
450
450
449
446
448
450
448
448
448
452
456
454
455
453
456
454
15
452
443
456
452
456
454
455
455
131
456
455
457
453
457
457
131
15
13

608
606
606
604
604
604
608
604
608
608
607
604
607
453
453
994
616
612
453
612
612
614
610
994
612
994
611
613
453
453
610
614
611
616
612
994
610
615
613
453
615
614
611
611
994
611
612
453
615
994
616
615
610
611
994
611
994
453
615
613
611
611
994
994
613
610
610
616
614
611
453
612
613
616
615
453
613
613
453
615
616
453
615
616
615
616
453
615
453
613
613
615
610
613
453
610
613
613
453
615
610
615
610
615
617
617
617
617
617
617
618
618
618
618
618
618
618
618
618
618
618
618
618
618
59
619
621
619
621
620
619
621
620
59
620
59
619
621
620
620
621
619
619
620
59
619
621
621
619
621
620
620
620
621
620
621
59
621
621
59
620
621
620
621
59
620
621
622
622
622
623
622
623
622
622
623
623
622
622
623
623
623
623
623
623
623
627
628
627
628
625
627
626
627
628
625
624
626
627
628
625
624
624
625
625
627
628
624
625
626
624
626
627
625
628
628
628
627
627
626
628
627
628
627
627
625
627
627
626
627
625
626
626
627
625
626
627
625
625
626
626
625
629
630
630
629
630
629
630
630
630
629

737
737
794
737
795
794
737
794
794
737
737
737
795
737
794
737
737
737
794
794
737
737
794
794
796
796
796
796
796
796
796
796
796
796
796
796
796
797
569
276
569
797
276
276
276
797
276
797
797
276
569
569
797
569
797
276
797
569
569
797
276
569
569
569
569
569
569
569
569
569
120
799
120
799
798
120
798
799
120
799
798
799
120
799
798
120
799
120
799
798
120
120
798
120
120
120
120
120
803
807
801
806
803
804
802
807
800
800
801
803
805
800
807
803
805
804
806
802
805
802
805
804
806
803
805
807
805
806
807
800
807
802
804
806
803
807
805
806
801
804
803
802
803
806
802
800
802
807
804
803
806
807
804
800
803
802
800
803
804
806
802
802
800
803
806
802
801
800
804
803
804
800
806
801
803
800
800
801
804
806
806
801
800
801
801
806
801
806
801
800
801
801
800
675
247
808
811
675
810
675
810
809
810
808
808
810
810
808
811
810
811
809
810
675
810
809
675
811
247
808
247
809
247
675
811
247
809
811
247
675
675
809
811
808
247
811
809
811
811
809
675
675
811
808
808
675
811
809
811
811


946
945
947
536
946
946
947
948
948
945
946
948
948
536
948
946
946
948
946
536
536
536
536
946
536
948
948
948
948
948
952
950
952
951
951
951
423
952
950
950
952
950
950
952
951
423
950
950
952
951
951
423
423
952
951
952
951
423
951
952
950
950
951
951
951
951
952
952
423
952
423
952
423
423
952
952
423
423
423
423
423
423
953
953
954
953
954
953
953
954
954
953
953
953
954
954
954
954
954
954
955
955
956
955
955
956
955
955
956
956
955
955
955
955
956
955
955
956
955
956
956
956
956
956
956
956
958
996
958
664
243
957
996
996
664
958
957
958
958
664
958
996
957
664
664
664
664
243
243
996
957
243
664
957
664
664
957
957
996
996
996
957
957
957
996
957
957
996
957
996
997
997
997
959
105
105
997
997
959
997
997
105
959
105
959
105
105
959
959
105
105
105
105
105
959
105
959
960
961
960
188
188
961
388
188
388
388
960
961
388
961
388
960
188
961
961
188
188
960
961
960
388
188
388
388
388
388
188
388
962
962
962
962
962
962
962
962
963
963
963
963
963
963
963
963
963
963
963
963
963


In [79]:
for d in list_visits_good:
    user_id = GetUser(d['ip'])
    category_id = GetCategoryGood(d['category'])
    if category_id == None:
        cursor.execute("""insert into category_goods (name) values(?)""",[(d['category'])])
        category_id = GetCategoryGood(d['category'])
    good_id =GetGood(d['good'],category_id)
    if good_id == None:
            cursor.execute("""insert into goods ( name, category_id) values(?,?)""",[(d['good']),(category_id)])
            good_id =GetGood(d['good'],category_id)
    cursor.execute("""insert into visits (datetime,category_id,good_id,user_id) values(?,?,?,?)""",[(d['datetime']),(category_id),(good_id),(user_id)])
    

In [101]:
for d in list_carts:
        user_id = GetUser(d['ip'])
        cursor.execute("""select id from cart_history where id = ? """,[(d['cart_id'])])
        if len(cursor.fetchall()) == 0:
            cursor.execute("""select good_id from visits  where user_id = ? and datetime < ? ORDER BY datetime desc""",[(user_id),(d['datetime'])])
            good_id = cursor.fetchall()[0][0]
            cursor.execute("""insert into cart_history (id,datetime,good_id,user_id,amount) values(?,?,?,?,?)""",[(d['cart_id']),(d['datetime']),(good_id),(user_id),(d['amount'])])

In [103]:
for d in list_pay:
    cursor.execute("""insert into purchases (datetime,cart_id) values(?,?)""",[(d['datetime']),(d['cart_id'])])

In [108]:
for d in list_success_pay:
    cursor.execute("""update purchases set status = true where cart_id = ?""",[(d['cart_id'])])